# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from unityagents import UnityEnvironment
import matplotlib.pyplot as plt
import numpy as np
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [4]:
script_path = os.getcwd()
env_path = os.path.join(script_path, "Banana_Windows_x86_64", "Banana_Windows_x86_64", "Banana.exe")
env = UnityEnvironment(file_name=env_path,seed=3)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [5]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
print(brain)

Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [6]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [0.         1.         0.         0.         0.41971061 0.
 1.         0.         0.         0.4629274  0.         0.
 0.         1.         0.         1.         0.         0.
 0.         0.22897126 0.         0.         0.         1.
 0.         0.         0.         1.         0.         0.1093323
 0.         1.         0.         0.         0.69851577 0.
 0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [10]:
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(QNetwork,self).__init__()
        fc2_size = action_size*2
        fc3_size = int(action_size*0.5)
        
        self.fc1 = nn.Linear(state_size, fc2_size)
        self.fc2 = nn.Linear(fc2_size, fc3_size)
        self.fc3 = nn.Linear(fc3_size, action_size)
        
    def forward(self,state):
        
        x = torch.from_numpy(state).float().to(device)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        
        return x

class DQNAgent():
    
    def __init__(self, settings):
        self.action_size = settings["action size"]
        self.state_size = settings["state size"]
        self.batch_size = settings["batch size"]
        self.update_freq = settings["update frequency"]
        self.update_counter = settings["update frequency"]
        
        self.qnetwork_local = QNetwork(state_size, action_size).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size).to(device)
        
        self.optimizer = optim.Adam(self.qnetwork_local.parameters())
        
        
        self.exp_mem = []
    
    
    def act(self, epsilon, state):
        if np.random.rand() <= epsilon:
            action = np.random.randint(self.action_size)
        else:
            actions = self.qnetwork_local(state)
            action = np.argmax(actions.cpu().data.numpy()).astype(int)

        return action
    
    def step(self, state, action, reward, next_state, done):
        
        exp = (state,action,reward,next_state,done)
        
        self.exp_mem.append(exp)
        
        # retrain
        if len(self.exp_mem) > self.batch_size and self.update_counter == 0:
            self.update_counter = self.update_freq
                        
            train_mems = set([])            
            while len(train_mems) < self.batch_size:
                train_mems.add(random.choice(self.exp_mem))
            
            self.learn(train_mems)
            
        else:
            self.update_counter -= 1
            
    
    def learn(self, experiences):
                   
        states, actions, rewards, next_states, dones = experiences


        # Get max predicted Q values (for next states) from target model
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        # Compute Q targets for current states 
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        # Get expected Q values from local model
        Q_expected = self.qnetwork_local(states).gather(1, actions)

        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # ------------------- update target network ------------------- #
        self.soft_update(self.qnetwork_local, self.qnetwork_target)
            
    
    def soft_update(self, local_model, target_model, tau=0.7):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

In [11]:
def DeepReinforcementLearning(train_mode, agent):
    score = 0                                             # initialize the score
    epsilon = 0.4
    
    scores = []
    if train_mode:
        max_episodes = 500
    else:
        max_episodes = 1
    
    for i_episode in range(max_episodes):
        env_info = env.reset(train_mode=True)[brain_name] # reset the environment
        state = env_info.vector_observations[0]            # get the current state
        while True:
            action = agent.act(epsilon, state)             # select an action
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished
            agent.step(state, action, reward, next_state, done)
            score += reward                                # update the score
            state = next_state                             # roll over the state to next time step
            if done:                                       # exit loop if episode finished
                break
        scores.append(score)
    
    if train_mode:
        plt.plot(scores)
        plt.ylabel('Score')
        plt.xlabel('Episode')
        plt.show()
        
        print("Final score: {}".format(score))
    

In [13]:
settings ={"update frequency": 4,
           "batch size": 32,
           "n_state": state_size,
           "n_action": action_size}

deep_q_agent = DQNAgent(settings)

DeepReinforcementLearning(train_mode=True, agent=deep_q_agent)
DeepReinforcementLearning(train_mode=False, agent=deep_q_agent)



KeyError: 'action size'

When finished, you can close the environment.

In [47]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```